In [31]:
import boto3
import mlflow.sagemaker as mfs
import json
app_name = "mlops-sagemaker"
execution_role_arn = "arn:aws:iam::544973878066:role/SageMakerS3Role"
#execution_role_arn = "arn:aws:iam::544973878066:policy/SageMakerS3BucketPolicy"
image_ecr_url = "544973878066.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:1.28.0"
region = "us-east-1"
s3_bucket_name = "mlops-sagemaker-runs-cap1"
experiment_id = "mlruns/1"
run_id = "273573d6f970410caf54aa15d59d2f84"
model_name = "log_reg_model"
model_uri = "s3://mlops-sagemaker-runs-cap1/mlruns/1/273573d6f970410caf54aa15d59d2f84/artifacts/log_reg_model/"
#model_uri = "s3://{}/{}/{}/artifacts/{}/".format(s3_bucket_name, experiment_id, run_id, model_name)
print(model_uri)

s3://mlops-sagemaker-runs-cap1/mlruns/1/273573d6f970410caf54aa15d59d2f84/artifacts/log_reg_model/


###### Run the deployment code.

In [32]:
mfs.deploy(app_name=app_name,
model_uri=model_uri,
execution_role_arn=execution_role_arn,
region_name=region,
image_url=image_ecr_url,
mode=mfs.DEPLOYMENT_MODE_CREATE)

2022/08/26 09:17:15 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2022/08/26 09:17:15 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2022/08/26 09:17:16 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-1-544973878066` already exists. Skipping creation.
2022/08/26 09:17:17 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': 'BGM11PVTXMZ1CM0V', 'HostId': 'sagxsv9RetdSS5foxx6EyZXIpF+LUCefv8Qtyc5+1XABX/0zW/A/mDCatXlNRzoiNXrfYmxBMis=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'sagxsv9RetdSS5foxx6EyZXIpF+LUCefv8Qtyc5+1XABX/0zW/A/mDCatXlNRzoiNXrfYmxBMis=', 'x-amz-request-id': 'BGM11PVTXMZ1CM0V', 'date': 'Fri, 26 Aug 2022 13:17:16 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2022/08/26 09:17:17 INFO mlflow.sagemaker: Creating new endpoint with name: mlops-sagemaker ...
2022/08/26 09:17:18 INFO mlflow.sagemaker: Created model with arn: arn:aws:sagemaker:us-east-1

('mlops-sagemaker', 'python_function')

In [33]:
def query(input_json):
    client = boto3.session.Session().client("sagemaker-runtime", region)
    response = client.invoke_endpoint(
    EndpointName=app_name,
    Body=input_json,
    ContentType='application/json; format=pandas-split',
    )
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    return preds

In [34]:
import pandas as pd
import mlflow
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
import numpy as np
data_path = "C:\\Users\\Anoops\\Documents\\Study\\Capstone\\creditcard.csv"
df = pd.read_csv(data_path)

normal = df[df.Class == 0].sample(frac=0.5, random_state=2020).reset_index(drop=True)
anomaly = df[df.Class == 1]
normal_train, normal_test = train_test_split(normal, test_size = 0.2, random_state = 2020)
anomaly_train, anomaly_test = train_test_split(anomaly, test_size = 0.2, random_state = 2020)
scaler = StandardScaler()
scaler.fit(pd.concat((normal, anomaly)).drop(["Time", "Class"], axis=1))

StandardScaler()

In [35]:
test = pd.concat((normal.iloc[:1900], anomaly.iloc[:100]))
true = test.Class
test = scaler.transform(test.drop(["Time", "Class"], axis=1))
preds = []
batch_size = 80
for f in range(25):
    print(f"Batch {f}", end=" - ")
    sample = pd.DataFrame(test[f*batch_size:(f+1)*batch_size]).to_json(orient="split")
    print("sample is ")
    print(sample)
    output = query(sample)
    resp = pd.DataFrame([output])
    preds = np.concatenate((preds, resp.values[0]))
    print(preds)
    print("Completed")
eval_acc = accuracy_score(true, preds)
eval_auc = roc_auc_score(true, preds)
print("Eval Acc", eval_acc)
print("Eval AUC", eval_auc)

Batch 0 - sample is 
{"columns":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79],"data":[[1.1108887323,-0.9781278999,-0.9060774366,-1.3955093902,-0.1638263697,1.0851054245,-1.1094798772,0.3770245036,-1.1594517589,1.4736019443,0.4662995562,-0.701687547,-0.7885465373,0.0812880965,0.0694434054,-1.1206468669,0.9660630591,-0.7006654589,-0.7669201918,-0.7120084991,-0.1357526202,0.2925611598,0.4148026231,-1.649467293,-0.6536265494,-0.1597408526,0.1017263601,-0.2291676389,-0.2885468328],[-0.2933943782,0.3832590781,0.0585323684,-1.4115824962,0.1918446843,-0.5279935006,0.3380141403,0.1327491965,-1.5319555982,-0.2475928898,-1.8101484189,-0.2860446737,0.9665032564,0.2857369204,-0.2339067097,-1.0933605256,-0.6166890412,1.5490242729,-

In [36]:
# mfs.delete(app_name=app_name,region_name=region)
